In [1]:
import pickle
import math
import time
import argparse
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.functional import gumbel_softmax
from utils import gumbel_softmax_3d
%matplotlib inline

In [2]:
def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    G = nx.from_dict_of_lists(data)
    return G

In [3]:
def train(args, G):
    bs = args.batch_size
    n = G.number_of_nodes()
    A = nx.to_numpy_matrix(G)
    A = torch.from_numpy(A)
    A = A.type(torch.float32)
    best_log = []
    A = A.cuda()

    # set parameters
    if torch.cuda.is_available():
        A = A.cuda()
        x = torch.randn(bs, n, 1, device='cuda')*1e-5
        x.requires_grad = True
    else:
        x = torch.randn(bs, n, 1, requires_grad=True)*1e-5

    # set optimizer
    optimizer = torch.optim.Adam([x], lr=args.lr)

    # training
    cost_arr = []
    for _ in range(args.iterations):
        optimizer.zero_grad()
        if torch.cuda.is_available():
            probs = torch.empty(bs, n, 2, device='cuda')
        else:
            probs = torch.empty(bs, n, 2)
        p = torch.sigmoid(x)
        probs[:, :, 0] = p.squeeze()
        probs[:, :, -1] = 1-probs[:, :, 0]
        logits = torch.log(probs+1e-10)
        s = gumbel_softmax_3d(logits, tau=args.tau, hard=args.hard)[:, :, 0]
        s = torch.unsqueeze(s, -1)  # size [bs, n, 1]
        cost = torch.sum(s)
        constraint = torch.sum((1-torch.transpose(s, 1, 2)) @ A @ (1-s))
        loss = cost + args.eta * constraint
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            constraint_ = torch.squeeze((1-torch.transpose(s, 1, 2)) @ A @ (1-s))
            constraint = constraint_.cpu().numpy()
            idx = np.argwhere(constraint == 0)  # select constraint=0

            
            cost_ = torch.sum(s, dim=1).squeeze()

            loss_ = cost_ + constraint_
            
            if len(idx) != 0:

                s = gumbel_softmax_3d(logits, tau=args.tau, hard=True)[:, :, 0]
                s = torch.unsqueeze(s, -1).cpu()  # size [bs, n, 1]
                cost_ = torch.sum(s, dim=1).squeeze()
                cost = torch.sum(s, dim=1)[idx.reshape(-1,)]
                # from size [bs, 1] select constrain=0
                
                if _ > 5000 and torch.min(cost) < min(cost_arr):
                    k = np.argwhere(torch.min(cost.cpu()))
                    
                    print('##################################################')
                    print('####################',_,'####################')
                    print(idx.squeeze())
                    print(cost_[idx.squeeze()])
                    print('##################################################')

                
                cost_arr.append(torch.min(cost.cpu()))
                
            #交叉变异
            if _ % 10000 == 0 and _ >= 10000:
                
                ratio = 1/4
                r = int(ratio * bs)
                m = 2e-3
                
                maxdx = torch.argsort(loss_, dim=0,descending=True).squeeze()
                mindx = torch.argsort(loss_, dim=0,descending=False).squeeze()

                #print('maxdx:',maxdx)
                #print('mindx:',mindx)
                L1 = random.sample(list(maxdx[0:r]), r)
                L2 = random.sample(list(mindx[0:r]), r)
                print('L1:',L1)
                #print('L2:',L2)
                #print(loss_[L1[0]])
                for i in range(r//2):
                    #print(L1[i])
                    #print('before:',nnn.pps.data[L1[i], : ,0])
                    #print(L2[i])
                    #print('father:',nnn.pps.data[L2[i], :, 0])
                    #print(L2[-(i+1)])
                    #print('mother:',nnn.pps.data[L2[-(i+1)], :, 0])
                    rand = torch.rand(n).cuda()
                    x.data[L1[i], : ,0] = torch.where(rand < 0.5, 
                                                            x.data[L2[i], :, 0], 
                                                            x.data[L2[r-1-i], :, 0])
                    x.data[L1[r-1-i], :, 0] = torch.where(rand < 0.5, 
                                                            x.data[L2[r-1-i], :, 0], 
                                                            x.data[L2[i], :, 0])
                    #print('after:',nnn.pps.data[L1[i], : ,0])
                    rand = torch.rand(n).cuda()
                    x.data[L1[i], : ,0] = torch.where(rand < m, 
                                                            x.data[L1[i], : ,0], 
                                                            torch.randn_like(x.data[L1[i], : ,0]) * 1e-5)
                    #count = torch.where(rand<m,torch.ones_like(rand),torch.zeros_like(rand))
                    #print('mutation_count:',torch.sum(count))
                
            if _ % 1000 == 0:
                print(_)
                if len(cost_arr) != 0:

                    print('constraint:',torch.sort(constraint_))
                    print('loss:',torch.sort(loss_))
                    print('# {}, cost: {}'.format(_, ((np.sort(cost_arr))[0:8])))
                else:
                    print('Failed!')

    return cost_arr

In [4]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(
        description='Solving MIS problems (with fixed tau in GS, parallel version)')
    parser.add_argument('--batch-size', type=int, default=128,
                        help='batch size (default: 128)')
    parser.add_argument('--data', type=str, default='citeseer',
                        help='data name (default: cora)')
    parser.add_argument('--tau', type=float, default=1.,
                        help='tau value in Gumbel-softmax (default: 1)')
    parser.add_argument('--hard', type=bool, default=True,
                        help='hard sampling in Gumbel-softmax (default: True)')
    parser.add_argument('--lr', type=float, default=1e-2,
                        help='learning rate (default: 1e-2)')
    parser.add_argument('--eta', type=float, default=3.,
                        help='constraint (default: 5)')
    parser.add_argument('--ensemble', type=int, default=10,
                        help='# experiments (default: 100)')
    parser.add_argument('--iterations', type=int, default=1000000,
                        help='# iterations in gradient descent (default: 20000)')
    parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')
    args = parser.parse_args(args=[])

    # torch.manual_seed(args.seed)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)

    # loading data
    G = load_data('./data/ind.' + args.data + '.graph')

    for i in range(args.ensemble):
        cost = train(args, G)
        if len(cost) != 0:
            print('# {}, cost: {}'.format(i, min(cost)))
        else:
            print('Failed!')


if __name__ == '__main__':
    main()

cuda


KeyboardInterrupt: 